--- Part Two ---
While The Historians begin working around the guard's patrol route, you borrow their fancy device and step outside the lab. From the safety of a supply closet, you time travel through the last few months and record the nightly status of the lab's guard post on the walls of the closet.

Returning after what seems like only a few seconds to The Historians, they explain that the guard's patrol area is simply too large for them to safely search the lab without getting caught.

Fortunately, they are pretty sure that adding a single new obstruction won't cause a time paradox. They'd like to place the new obstruction in such a way that the guard will get stuck in a loop, making the rest of the lab safe to search.

To have the lowest chance of creating a time paradox, The Historians would like to know all of the possible positions for such an obstruction. The new obstruction can't be placed at the guard's starting position - the guard is there right now and would notice.

In the above example, there are only 6 different positions where a new obstruction would cause the guard to get stuck in a loop. The diagrams of these six situations use O to mark the new obstruction, | to show a position where the guard moves up/down, - to show a position where the guard moves left/right, and + to show a position where the guard moves both up/down and left/right.

In [396]:
guard_characters = ["^", ">", "V", "<"]
pathing_chars = ["-", "|", "+"]
blocked_chars = [".", "#"]
initial_grid = [
    "....#.....",
    ".........#",
    "..........",
    "..#.......",
    ".......#..",
    "..........",
    ".#..^.....",
    "........#.",
    "#.........",
    "......#...",
]
initial_guard_loc_x = 0
initial_guard_loc_y = 0
initial_guard_direction = "^"



In [397]:
def get_file_content():
    with open("day-6-input.txt", "r") as file:
        # Read all lines and strip any trailing newlines or spaces
        grid = [line.strip() for line in file]
    return grid

initial_grid = get_file_content()

In [398]:
for y in range(len(initial_grid)):
    for x in range(len(initial_grid[y])):
        if initial_grid[y][x] in guard_characters:
            initial_guard_loc_x = x
            initial_guard_loc_y = y
            initial_guard_direction = initial_grid[y][x]


def get_movement_dir(guard_dir):
    match guard_dir:
        case "^":
            return (0, -1, "|")
        case ">":
            return (1, 0, "-")
        case "V":
            return (0, 1, "|")
        case "<":
            return (-1, 0, "-")
        case _:
            return (0, 0, ".")


def get_lookahead(guard_loc_x, guard_loc_y, offset, grid, reference_char=["#"]):
    value = grid[guard_loc_y + offset[1]][guard_loc_x + offset[0]]
    return value not in reference_char


def get_next_direction(guard_direction):
    index_of_direction = guard_characters.index(guard_direction)
    try:
        return guard_characters[index_of_direction + 1]
    except IndexError:
        return guard_characters[0]


def mutate_grid(x, y, grid, char):
    mutable_grid = [list(row) for row in grid]
    mutable_grid[y][x] = char
    grid = ["".join(row) for row in mutable_grid]
    return grid


def change_to_tracked(guard_loc_x, guard_loc_y, grid, direction_character):
    current_char = grid[guard_loc_y][guard_loc_x]
    if current_char in pathing_chars:
        direction_character = "+"
    grid = mutate_grid(guard_loc_x, guard_loc_y, grid, direction_character)
    return grid


def print_grid(grid):
    for line in grid:
        print(line)
    print("-----------")


def move_forward(guard_loc_x, guard_loc_y, guard_direction, grid, loop_references):
    offset = (0, 0, ".")
    can_move_forward = False
    has_turned = False
    while can_move_forward == False:
        offset = get_movement_dir(guard_direction)
        can_move_forward = get_lookahead(
            guard_loc_x, guard_loc_y, offset, grid, ["#", "O"]
        )
        if can_move_forward == False:
            has_turned = True
            guard_direction = get_next_direction(guard_direction)
            if (guard_loc_y, guard_loc_x, guard_direction) in loop_references:
                raise Exception("Closed loop")
            else:
                loop_references.append((guard_loc_y, guard_loc_x, guard_direction))

    grid = change_to_tracked(
        guard_loc_x, guard_loc_y, grid, "+" if has_turned else offset[2]
    )
    guard_loc_y = guard_loc_y + offset[1]
    guard_loc_x = guard_loc_x + offset[0]
    return (guard_loc_x, guard_loc_y, guard_direction, grid, loop_references)

In [399]:
import time


def run_full_analysis(guard_loc_x, guard_loc_y, guard_direction, grid):
    loop_references = []
    while True:
        try:
            guard_loc_x, guard_loc_y, guard_direction, grid, loop_references = (
                move_forward(
                    guard_loc_x, guard_loc_y, guard_direction, grid, loop_references
                )
            )
        except IndexError:
            dir_char = get_movement_dir(guard_direction)
            grid = change_to_tracked(guard_loc_x, guard_loc_y, grid, dir_char[2])
            break
        except Exception as e:
            print("CLOSED LOOP")
            return (1, grid)

    return (0, grid)

In [400]:
end_count = 0
end_count, check_grid = run_full_analysis(
    initial_guard_loc_x, initial_guard_loc_y, initial_guard_direction, initial_grid
)

print_grid(check_grid)

...........#....................#......#.....................#.#...........#......................................................
.....................................#....#.................#...............................#.....................................
..#..#.......#.............................................#.#.........................#......................#..............#....
.........................#.....#...............#.................#................................................................
.......#..........#........#.#.+-----------------------------------------+#....#.....................#..........##..#.............
.........#.....................|...............................#.........|....................................................#...
.....#...+---------------------+-----------------------------------+#....|...........##.....#...........#.......#.........#.......
.......#.|.......#......#......|........#..........................|.#...|.....#...

In [401]:
for y in range(len(initial_grid)):
    for x in range(len(initial_grid[y])):
        if check_grid[y][x] not in blocked_chars:
            grid, guard_loc_x, guard_loc_y, guard_direction = (
                initial_grid,
                initial_guard_loc_x,
                initial_guard_loc_y,
                initial_guard_direction,
            )
            grid = mutate_grid(x, y, grid, "O")

            outcome, grid = run_full_analysis(
                guard_loc_x, guard_loc_y, guard_direction, grid
            )
            end_count += outcome
            print(f"completed run {y}, {x}")

print(end_count)

CLOSED LOOP
completed run 4, 31
CLOSED LOOP
completed run 4, 32
completed run 4, 33
completed run 4, 34
CLOSED LOOP
completed run 4, 35
completed run 4, 36
completed run 4, 37
CLOSED LOOP
completed run 4, 38
CLOSED LOOP
completed run 4, 39
CLOSED LOOP
completed run 4, 40
completed run 4, 41
CLOSED LOOP
completed run 4, 42
completed run 4, 43
completed run 4, 44
CLOSED LOOP
completed run 4, 45
CLOSED LOOP
completed run 4, 46
completed run 4, 47
completed run 4, 48
completed run 4, 49
completed run 4, 50
completed run 4, 51
completed run 4, 52
completed run 4, 53
CLOSED LOOP
completed run 4, 54
completed run 4, 55
CLOSED LOOP
completed run 4, 56
CLOSED LOOP
completed run 4, 57
CLOSED LOOP
completed run 4, 58
CLOSED LOOP
completed run 4, 59
completed run 4, 60
CLOSED LOOP
completed run 4, 61
CLOSED LOOP
completed run 4, 62
CLOSED LOOP
completed run 4, 63
CLOSED LOOP
completed run 4, 64
CLOSED LOOP
completed run 4, 65
CLOSED LOOP
completed run 4, 66
CLOSED LOOP
completed run 4, 67
complete

KeyboardInterrupt: 